In [1]:
# Forest_Depth_ED_Mapping.py
# Local 3x3 neighbor mosaic + halo Euclidean forest depth (meters).
# 1 = forest, 0 = non-forest. Output: float32 meters; non-forest set to 0.
#
# How to run:
#   python Forest_Depth_ED_Mapping.py
#
# Dependencies:
#   pip install rasterio numpy scipy tqdm pyproj
# (pyproj is optional but improves degree->meter accuracy.)
#
# Notes:
# - Works whether rasters are in a projected CRS (meters) or in degrees (EPSG:4326 etc.).
# - For degree CRS, meters-per-pixel is computed using local latitude (pyproj.Geod if available).
# - Uses only the target tile + its 3×3 neighborhood and a pixel halo to avoid global VRTs.

import os, math, time, warnings, gc
from datetime import datetime
from rasterio.windows import Window
import numpy as np
import rasterio
from rasterio.transform import array_bounds
from affine import Affine
from scipy.ndimage import distance_transform_edt
from tqdm import tqdm

# --------- USER SETTINGS (edit these paths if needed) ---------
INPUT_DIR  = r"/mnt/cephfs-mount/hangkai/2010"
OUTPUT_DIR = r"/mnt/cephfs-mount/hangkai/2010_depth"

HALO_PX = 2048                # Pixel halo around each tile (e.g., 256 / 512 / 1024)
SUBTRACT_HALF_PIXEL = True    # Better approximates distance to boundary vs pixel center
OVERWRITE = True             # Skip tiles that already have outputs when False
LIMIT_TILES = None            # e.g., 5 to test only first 5 tiles; set None for all
COMPRESSION = "LZW"           # "LZW" or "DEFLATE" etc.

# --------- Optional: accurate meters-per-degree via pyproj ---------
try:
    from pyproj import Geod
    GEOD = Geod(ellps="WGS84")
except Exception:
    GEOD = None


# ====================== Utilities ======================
def now_str():
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

def list_tifs(input_dir):
    tifs = [os.path.join(input_dir, n) for n in os.listdir(input_dir)
            if n.lower().endswith((".tif", ".tiff"))]
    tifs.sort()
    return tifs

def build_tile_index(tif_paths):
    """Gather per-tile metadata used for neighbor search and merging."""
    index = []
    crs0 = None
    for p in tqdm(tif_paths, desc="Indexing tiles", unit="tile"):
        with rasterio.open(p) as ds:
            h, w = ds.height, ds.width
            btm, lft, top, rgt = array_bounds(h, w, ds.transform)  # (ymin, xmin, ymax, xmax)
            if abs(ds.transform.b) > 1e-9 or abs(ds.transform.d) > 1e-9:
                print(f"[{now_str()}] WARNING: {os.path.basename(p)} has rotation/shear in transform.")
            if crs0 is None:
                crs0 = ds.crs
            elif ds.crs != crs0:
                print(f"[{now_str()}] WARNING: CRS differs for {os.path.basename(p)}.")
            index.append({
                "path": p,
                "name": os.path.basename(p),
                "width": w,
                "height": h,
                "transform": ds.transform,
                "bounds": (lft, btm, rgt, top),             # (L, B, R, T)
                "center": ((lft+rgt)/2.0, (btm+top)/2.0),   # (cx, cy)
                "xres": abs(ds.transform.a),
                "yres": abs(ds.transform.e),
                "crs": ds.crs
            })
    # Quick lookup by path
    path2meta = {m["path"]: m for m in index}
    return index, path2meta

def intersects(b1, b2):
    L1,B1,R1,T1 = b1
    L2,B2,R2,T2 = b2
    return (L1 < R2) and (R1 > L2) and (B1 < T2) and (T1 > B2)

def meters_per_pixel_from_shape(transform, width, height, crs, center_lat=None):
    """
    Robustly return (yres_m, xres_m), guaranteeing strictly positive numbers.
    Works for projected and geographic CRS; tolerates bad/rounded transforms.
    """
    # Try to read pixel size from transform first
    xres_deg_or_m = abs(getattr(transform, "a", 0.0))
    yres_deg_or_m = abs(getattr(transform, "e", 0.0))

    # Helper: fallback to degrees-per-pixel via array bounds if transform looks bad
    def deg_per_px_via_bounds():
        ymin, xmin, ymax, xmax = array_bounds(height, width, transform)  # (ymin,xmin,ymax,xmax)
        dx_deg = (xmax - xmin) / max(1, width)
        dy_deg = (ymax - ymin) / max(1, height)
        return abs(dy_deg), abs(dx_deg), (ymax + ymin) / 2.0

    # Projected CRS → return as meters (guard for US feet)
    if crs and crs.is_projected:
        # crude feet sniff
        try:
            wkt = crs.to_wkt().lower()
            feet = ("foot" in wkt) or ("feet" in wkt) or ("us-ft" in wkt)
        except Exception:
            feet = False

        y_m = yres_deg_or_m
        x_m = xres_deg_or_m

        # If transform looks bogus, try to derive via bounds (still in meters because projected CRS)
        if y_m <= 0 or x_m <= 0:
            ymin, xmin, ymax, xmax = array_bounds(height, width, transform)
            x_m = abs((xmax - xmin) / max(1, width))
            y_m = abs((ymax - ymin) / max(1, height))

        if feet:
            y_m *= 0.3048
            x_m *= 0.3048

        # Final guard-rails
        eps = 1e-6
        return (max(y_m, eps), max(x_m, eps))

    # Geographic CRS → convert deg/px to m/px
    # If transform looks okay, use its a/e as deg/px; else derive from bounds
    if (xres_deg_or_m <= 0) or (yres_deg_or_m <= 0):
        y_deg_per_px, x_deg_per_px, cen_lat = deg_per_px_via_bounds()
        xres_deg_or_m = x_deg_per_px
        yres_deg_or_m = y_deg_per_px
        center_lat = cen_lat
    else:
        if center_lat is None:
            ymin, xmin, ymax, xmax = array_bounds(height, width, transform)
            center_lat = (ymax + ymin) / 2.0

    # Convert degrees to meters
    if GEOD is not None:
        ymin, xmin, ymax, xmax = array_bounds(height, width, transform)
        center_lon = (xmin + xmax) / 2.0
        _, _, dx_m = GEOD.inv(center_lon, center_lat, center_lon + xres_deg_or_m, center_lat)
        _, _, dy_m = GEOD.inv(center_lon, center_lat, center_lon, center_lat + yres_deg_or_m)
        x_m = abs(dx_m)
        y_m = abs(dy_m)
    else:
        phi = math.radians(center_lat)
        m_per_deg_lat = 111132.92 - 559.82*math.cos(2*phi) + 1.175*math.cos(4*phi) - 0.0023*math.cos(6*phi)
        m_per_deg_lon = 111412.84*math.cos(phi) - 93.5*math.cos(3*phi) + 0.118*math.cos(5*phi)
        x_m = xres_deg_or_m * abs(m_per_deg_lon)
        y_m = yres_deg_or_m * abs(m_per_deg_lat)

    # Absolute final safety net
    eps = 1e-6
    return (max(y_m, eps), max(x_m, eps))



def process_tile_local3x3(tile_meta, all_meta, halo_px, out_path,
                          subtract_half_pixel=True, compress="LZW"):
    """
    Forest depth for one tile using a 3×3 neighborhood with integer-grid placement (no resampling).

    Assumptions
    -----------
    - All tiles share CRS, pixel size, and are north-up (no rotation/shear).
    - Tiles are perfectly aligned on the same grid.
    """
    t0 = time.perf_counter()

    # --- Debug toggles (adjust as you like) ---
    DEBUG_SAVE = False                       # set True to write debug rasters per tile
    DEBUG_TILES = {"50E_120N.tif"}           # only save for these basenames

    def _save_debug(arr, profile, path):
        if not DEBUG_SAVE:
            return
        os.makedirs(os.path.dirname(path), exist_ok=True)
        prof = profile.copy()
        prof.update(driver="GTiff", dtype="float32", count=1, nodata=0.0,
                    compress="LZW", predictor=3, tiled=True,
                    blockxsize=512, blockysize=512,
                    bigtiff="IF_SAFER", interleave="band")
        with rasterio.open(path, "w", **prof) as dst:
            dst.write(arr.astype(np.float32, copy=False), 1)

    # ---- Center tile basics ----
    W, H = tile_meta["width"], tile_meta["height"]
    tx, crs = tile_meta["transform"], tile_meta["crs"]

    # Safety: north-up only
    if abs(tx.b) > 1e-9 or abs(tx.d) > 1e-9:
        raise RuntimeError("Center tile has rotation/shear; integer-grid placement requires north-up.")

    # ---- Target grid (expanded outward by HALO_PX) ----
    DST_W  = W + 2*halo_px
    DST_H  = H + 2*halo_px
    dst_transform = tx * Affine.translation(-halo_px, -halo_px)

    # Target mask: 0 = non-forest, 1 = forest
    mosaic_u8 = np.zeros((DST_H, DST_W), dtype=np.uint8)

    # ---- Load center tile once (for final mask + profile; short-circuit if empty) ----
    with rasterio.open(tile_meta["path"]) as src_center:
        center_profile = src_center.profile
        center_arr = src_center.read(1, masked=True).filled(0)  # robust to nodata

    name_for_debug = tile_meta["name"]
    DEBUG_SAVE = DEBUG_SAVE and (name_for_debug in DEBUG_TILES)

    center_has_forest = bool((center_arr == 1).any())
    if not center_has_forest:
        core = np.zeros((H, W), dtype=np.float32)
        center_profile.update(
            driver="GTiff", dtype="float32", count=1, nodata=0.0,
            compress=compress, predictor=3, tiled=True, blockxsize=512, blockysize=512,
            bigtiff="IF_SAFER", interleave="band"
        )
        os.makedirs(os.path.dirname(out_path), exist_ok=True)
        with rasterio.open(out_path, "w", **center_profile) as dst:
            dst.write(core, 1)
        del center_arr, core
        gc.collect()
        return time.perf_counter() - t0

    # ---- Map neighbors to grid offsets using upper-left coordinates ----
    cx0, cy0 = tx.c, tx.f  # UL of center tile
    a = tx.a               # col scale (>0)
    e = tx.e               # row scale (<0 for north-up)

    grid = {}
    for m in all_meta:
        mt = m["transform"]
        if m["crs"] != crs: continue
        if abs(mt.b) > 1e-9 or abs(mt.d) > 1e-9: continue
        if abs(mt.a - a) > 1e-9 or abs(mt.e - e) > 1e-9: continue

        nx0, ny0 = mt.c, mt.f
        dc = (nx0 - cx0) / a   # columns offset in pixels
        dr = (ny0 - cy0) / e   # rows offset in pixels (e < 0)

        # --- tolerant neighbor snapping (avoids losing neighbors due to tiny misalignments)
        tol = 0.05  # allow up to 5% of a tile misalignment
        dx_f = dc / W   # offset in "tiles", e.g., -1, 0, +1 ideally
        dy_f = dr / H

        dx_step = int(round(dx_f))
        dy_step = int(round(dy_f))

        # skip if misalignment exceeds tolerance (i.e., not near an integer tile step)
        if abs(dx_f - dx_step) > tol or abs(dy_f - dy_step) > tol:
            continue

        # still enforce 3x3 neighborhood limits
        if dx_step < -1 or dx_step > 1 or dy_step < -1 or dy_step > 1:
            continue
            
        grid[(dy_step, dx_step)] = m

    grid.setdefault((0, 0), tile_meta)

    # ---- Paste each neighbor onto the target canvas (integer indices only) ----
    def paste_neighbor_exact(src_path, dy_step, dx_step):
        dr0 = halo_px + dy_step * H
        dc0 = halo_px + dx_step * W
        if dr0 >= DST_H or dc0 >= DST_W or dr0 + H <= 0 or dc0 + W <= 0:
            return
        sr0 = max(0, -dr0)
        sc0 = max(0, -dc0)
        dr0c = max(0, dr0)
        dc0c = max(0, dc0)
        hh = min(H - sr0, DST_H - dr0c)
        ww = min(W - sc0, DST_W - dc0c)
        if hh <= 0 or ww <= 0:
            return
        
        with rasterio.open(src_path, sharing=False) as src:
            patch = src.read(1, window=Window(sc0, sr0, ww, hh))
        patch = (patch == 1).astype(np.uint8)
        view = mosaic_u8[dr0c:dr0c+hh, dc0c:dc0c+ww]
        if patch.shape != view.shape:
            h_eff = min(patch.shape[0], view.shape[0])
            w_eff = min(patch.shape[1], view.shape[1])
            patch = patch[:h_eff, :w_eff]
            view  = mosaic_u8[dr0c:dr0c+h_eff, dc0c:dc0c+w_eff]
        np.maximum(view, patch, out=view)

    order = [(0,0), (-1,0),(+1,0),(0,-1),(0,+1),(-1,-1),(-1,+1),(+1,-1),(+1,+1)]
    for dy_step, dx_step in order:
        m = grid.get((dy_step, dx_step))
        if m:
            paste_neighbor_exact(m["path"], dy_step, dx_step)

    # DEBUG: save mosaic mask
    if DEBUG_SAVE:
        _save_debug((mosaic_u8 == 1).astype(np.float32), center_profile,
                    os.path.join(os.path.dirname(out_path), f"{os.path.splitext(name_for_debug)[0]}_dbg_mosaic_mask.tif"))

    # ---- EDT on merged mask ----
    # Define forest_mask BEFORE EDT and drop mosaic to free RAM
    forest_mask = (mosaic_u8 == 1)
    del mosaic_u8; gc.collect()

    # Guard: if mosaic is all forest (no background), write zeros and return
    if not np.any(~forest_mask):
        core = np.zeros((H, W), dtype=np.float32)
        profile = center_profile
        profile.update(
            driver="GTiff", dtype="float32", count=1, nodata=0.0,
            compress=compress, predictor=3, tiled=True, blockxsize=512, blockysize=512,
            bigtiff="IF_SAFER", interleave="band"
        )
        os.makedirs(os.path.dirname(out_path), exist_ok=True)
        with rasterio.open(out_path, "w", **profile) as dst:
            dst.write(core, 1)
        del forest_mask, core, center_arr
        gc.collect()
        return time.perf_counter() - t0

    # Pixel spacing (meters) from the target grid
    ymin, xmin, ymax, xmax = array_bounds(DST_H, DST_W, dst_transform)
    center_lat = (ymax + ymin) / 2.0
    yres_m, xres_m = meters_per_pixel_from_shape(
        dst_transform, DST_W, DST_H, crs, center_lat
    )

    # absolute guarantees
    if not np.isfinite(yres_m) or yres_m <= 0: yres_m = 1e-6
    if not np.isfinite(xres_m) or xres_m <= 0: xres_m = 1e-6

    # Distance transform with anisotropic sampling (meters)
    dt64 = distance_transform_edt(forest_mask, sampling=(yres_m, xres_m))
    dist_chunk = np.nan_to_num(dt64, nan=0.0, posinf=0.0, neginf=0.0).astype(np.float32, copy=False)
    del dt64, forest_mask; gc.collect()

    if subtract_half_pixel:
        half_min = 0.5 * min(yres_m, xres_m)
        if not np.isfinite(half_min) or half_min < 0:
            half_min = 0.0
        np.maximum(dist_chunk - half_min, 0.0, out=dist_chunk)

    # DEBUG: entire halo result (optional)
    if DEBUG_SAVE:
        _save_debug(dist_chunk, center_profile,
                    os.path.join(os.path.dirname(out_path),
                                 f"{os.path.splitext(name_for_debug)[0]}_dbg_dist_full.tif"))

    # ---- Crop the center tile core (exactly H×W at (halo,halo)) ----
    core = dist_chunk[halo_px:halo_px+H, halo_px:halo_px+W]
    assert core.shape == (H, W)
    del dist_chunk; gc.collect()

    # ---- Final mask: keep values only where center tile itself is forest ----
    core = np.where(center_arr == 1, core, 0.0).astype(np.float32)

    # DEBUG: post-mask stats
    if DEBUG_SAVE:
        _save_debug((center_arr == 1).astype(np.float32), center_profile,
                    os.path.join(os.path.dirname(out_path), f"{os.path.splitext(name_for_debug)[0]}_dbg_center_mask.tif"))
        _save_debug(core, center_profile,
                    os.path.join(os.path.dirname(out_path), f"{os.path.splitext(name_for_debug)[0]}_dbg_core_result.tif"))

    del center_arr; gc.collect()

    # ---- Sanitize: clamp and write ----
    MAX_VAL = 1_000_000.0
    core = np.nan_to_num(core, nan=0.0, posinf=0.0, neginf=0.0).astype(np.float32, copy=False)
    np.clip(core, 0.0, MAX_VAL, out=core)

    profile = center_profile
    profile.update(
        driver="GTiff", dtype="float32", count=1, nodata=0.0,
        compress=compress, predictor=3, tiled=True, blockxsize=512, blockysize=512,
        bigtiff="IF_SAFER", interleave="band"
    )
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    with rasterio.open(out_path, "w", **profile) as dst:
        dst.write(core, 1)

    del core
    gc.collect()
    return time.perf_counter() - t0


# ====================== Main Routine ======================
def main():
    os.environ.setdefault("GDAL_PAM_ENABLED", "NO")  # no .aux.xml
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    tif_list = list_tifs(INPUT_DIR)
    if not tif_list:
        print(f"[{now_str()}] No GeoTIFFs found in: {INPUT_DIR}")
        return

    if LIMIT_TILES is not None:
        tif_list = tif_list[:int(LIMIT_TILES)]

    print(f"[{now_str()}] Mode: LOCAL 3x3 + HALO")
    print(f"  Input : {INPUT_DIR}")
    print(f"  Output: {OUTPUT_DIR}")
    print(f"  Files : {len(tif_list)}")
    print(f"  Halo (px): {HALO_PX}")
    print(f"  Subtract half pixel: {SUBTRACT_HALF_PIXEL}\n")

    # Build metadata index once
    index, path2meta = build_tile_index(tif_list)

    total_start = time.perf_counter()
    failures = 0

    for p in tqdm(tif_list, desc="Processing tiles", unit="tile"):
        name = os.path.basename(p)
        out_name = os.path.splitext(name)[0] + "_depth_m.tif"
        out_path = os.path.join(OUTPUT_DIR, out_name)

        if (not OVERWRITE) and os.path.exists(out_path):
            tqdm.write(f"[{now_str()}] Skip (exists): {out_name}")
            continue

        try:
            tile_meta = path2meta[p]
            dt = process_tile_local3x3(
                tile_meta, index, HALO_PX, out_path,
                subtract_half_pixel=SUBTRACT_HALF_PIXEL,
                compress=COMPRESSION
            )
            tqdm.write(f"[{now_str()}] Done {name} in {dt:.2f} s")
        except Exception as e:
            failures += 1
            tqdm.write(f"[{now_str()}] ERROR {name}: {e}")

    total_s = time.perf_counter() - total_start
    print(f"\n[{now_str()}] All done in {total_s/60.0:.2f} min. Failures: {failures}")


if __name__ == "__main__":
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        main()

[2025-09-24 05:15:42] Mode: LOCAL 3x3 + HALO
  Input : /mnt/cephfs-mount/hangkai/2010
  Output: /mnt/cephfs-mount/hangkai/2010_depth
  Files : 259
  Halo (px): 2048
  Subtract half pixel: True



Indexing tiles: 100%|█| 259/259 [00:43<00:00,  5.90tile/
Processing tiles:   0%| | 1/259 [06:52<29:32:43, 412.26s

[2025-09-24 05:23:18] Done 00N_000E.tif in 412.26 s


Processing tiles:   1%| | 2/259 [13:03<27:41:40, 387.94s

[2025-09-24 05:29:29] Done 00N_010E.tif in 370.90 s


...

[2025-09-25 06:15:27] Done 80N_150W.tif in 389.63 s


Processing tiles: 100%|▉| 258/259 [25:04:16<05:33, 333.5

[2025-09-25 06:20:42] Done 80N_160W.tif in 314.99 s


Processing tiles: 100%|█| 259/259 [25:09:33<00:00, 349.7

[2025-09-25 06:25:59] Done 80N_170W.tif in 317.49 s

[2025-09-25 06:25:59] All done in 1509.56 min. Failures: 0
